In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -Uq /kaggle/input/llm-whls/bitsandbytes-0.41.1-py3-none-any.whl
!pip install -Uq /kaggle/input/llm-whls/peft-0.4.0-py3-none-any.whl
!pip install -Uq /kaggle/input/hf-llms-whls/transformers-4.38.2-py3-none-any.whl
!pip install -Uq /kaggle/input/llm-accelerate-whl/accelerate-0.22.0-py3-none-any.whl

In [3]:
# !pip install accelerate

In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,AutoConfig
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from peft import PeftConfig, PeftModel,LoraConfig


2024-04-06 17:37:04.605808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-06 17:37:04.605909: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-06 17:37:04.732744: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
input_token_len = 512
output_token_len = 100

In [6]:
df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")

df


,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [7]:
model_path = "/kaggle/input/phi/transformers/1/1"  
adapter_path = "/kaggle/input/phi2-public-data-sft-adapter/pytorch/public-data-sft/1/phi2_public_data_sft"


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_path,trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


In [10]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",)

In [11]:
# compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)


In [12]:
model = AutoModelForCausalLM.from_pretrained(model_path,trust_remote_code=True)
# model = PeftModel.from_pretrained(model, adapter_path)

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
def get_peft_model(original_model, peft_config):
    # Your implementation here to create or modify the original_model using peft_config
    modified_model = model  # For illustration, you need to actually modify the model
    return modified_model  # Return the modified or new model object

# Call the get_peft_model function with the original model and peft_config parameters
model = get_peft_model(model, peft_config)

# Check if model is not None before calling methods on it
if model is not None:
    model.to(device)
    model.eval()
    print('Model loaded!')
else:
    print('Failed to load model!')

Model loaded!


In [14]:
def text_generate(ori_text, rew_text, model, tokenizer, input_max_len=512, output_len=20, device='cuda'):
    prompt = f"Instruct: Original Text:{ori_text}\nRewritten Text:{rew_text}\nWrite a prompt that was likely given to the LLM to rewrite original text to rewritten text.\nOutput:"
    inputs = tokenizer(prompt, max_length=input_max_len, truncation=True, return_tensors="pt", return_attention_mask=False)
    
    input_token_len = len(inputs.input_ids[0])
    inputs = {k:v.to(device) for k,v in inputs.items()}
    max_len = input_token_len + output_len
    outputs = model.generate(**inputs,
                         do_sample=False,
                         max_length=max_len,
                         pad_token_id=tokenizer.pad_token_id,
                         )
    text = tokenizer.batch_decode(outputs,skip_special_tokens=True,clean_up_tokenization_spaces=False)[0]
    start_index = text.find('Output:')
    generated_text = text[start_index+len('Output:'):].strip()
    return generated_text

In [15]:
mean_prompt = "'Please improve the following text using the writing style of, maintaining the original meaning but altering the tone, diction, and stylistic elements to match the new style.Enhance the clarity, elegance, and impact of the following text by adopting the writing style of , ensuring the core message remains intact while transforming the tone, word choice, and stylistic features to align with the specified style.'"

In [16]:
rewrite_prompts = []

In [17]:
for i, row in tqdm(df.iterrows(), total=len(df)):
    prompt = mean_prompt
    try:
        prompt = text_generate(row['original_text'],
                               row['rewritten_text'],
                               model,
                               tokenizer,
                               input_token_len,
                               output_token_len,
                               device,
                              )
    except:
        pass
        
    rewrite_prompts.append(prompt)

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
df['rewrite_prompt'] = rewrite_prompts

In [19]:
sub_df = df[['id', 'rewrite_prompt']]
sub_df.to_csv('submission.csv', index=False)